In [3]:
from datasets import load_dataset
data_path = "rlhn/remove-680K"
ds = load_dataset(data_path, cache_dir=None, split="train")
ds

Dataset({
    features: ['query_id', 'query', 'positive_passages', 'negative_passages', 'subset'],
    num_rows: 323622
})

In [4]:
ds[0]


{'query_id': 'a4e1b4101791ce83a44cd0a6ed60a0fd',
 'query': 'what is a gdb file gis',
 'positive_passages': [{'docid': 'f4c816cbfbf1aaec0a52a1b5cdf77e20',
   'text': '- What is a file geodatabase? A file geodatabase is a collection of files in a folder on disk that can store, query, and manage both spatial and nonspatial data. You create a file geodatabase in ArcGIS. File geodatabases are made up of seven system tables plus user data. 1  User data can be stored in the following types of datasets: 2  Feature class. 3  Feature dataset. 4  Mosaic dataset. 5  Raster catalog. 6  Raster dataset. 7  Schematic dataset. 8  Table (nonspatial) 9  Toolboxes.',
   'title': ''}],
 'negative_passages': [{'docid': '649a9c2e576fd07bd9e2768108ca7c2d',
   'text': '- GemFire provides the same basic functionality of a database: data access, querying, indexing, server-side functions, transactions and archival persistence. Because GFE is designed to run in the memory space of applications, it exposes data as 

In [5]:
def transform(example):
    result = {
        "anchor": example["query"],
        "positive": example["positive_passages"][0]["text"],
    }
    for i in range(7):
        if i < len(example["negative_passages"]):
            result[f"negative_{i+1}"] = example["negative_passages"][i]["text"]
        else:
            result[f"negative_{i+1}"] = ""
    return result

ds_transformed = ds.map(transform, remove_columns=ds.column_names, num_proc=8)
ds_transformed

Map (num_proc=8): 100%|██████████| 323622/323622 [00:11<00:00, 27283.24 examples/s]


Dataset({
    features: ['anchor', 'positive', 'negative_1', 'negative_2', 'negative_3', 'negative_4', 'negative_5', 'negative_6', 'negative_7'],
    num_rows: 323622
})

In [6]:
ds_transformed.save_to_disk("/data_x/EMBEDDING/DATA/SFT/v2-filtered-new/rlhn_remove_680k_processed")

Saving the dataset (3/3 shards): 100%|██████████| 323622/323622 [00:15<00:00, 21179.78 examples/s] 


---
### ko, en 합치기

In [7]:
import os

from datasets import load_from_disk

save_dir = "/data_x/yjoonjang/PAPERS/Anchor-Embedding/DATA"
os.makedirs(save_dir, exist_ok=True)

ds_ko_path = "/data_x/EMBEDDING/DATA/SFT/v2-filtered-new/train_ko_250625_KURE-v1_hn32_final"
ds_en_path = "/data_x/EMBEDDING/DATA/SFT/v2-filtered-new/rlhn_remove_680k_processed"

ds_ko = load_from_disk(ds_ko_path)
ds_en = load_from_disk(ds_en_path)

print(ds_ko)
print(ds_en)

Dataset({
    features: ['anchor', 'positive', 'negative_1', 'negative_2', 'negative_3', 'negative_4', 'negative_5', 'negative_6', 'negative_7', 'negative_8', 'negative_9', 'negative_10', 'negative_11', 'negative_12', 'negative_13', 'negative_14', 'negative_15', 'negative_16', 'negative_17', 'negative_18', 'negative_19', 'negative_20', 'negative_21', 'negative_22', 'negative_23', 'negative_24', 'negative_25', 'negative_26', 'negative_27', 'negative_28', 'negative_29', 'negative_30', 'negative_31', 'negative_32', 'label'],
    num_rows: 1539300
})
Dataset({
    features: ['anchor', 'positive', 'negative_1', 'negative_2', 'negative_3', 'negative_4', 'negative_5', 'negative_6', 'negative_7'],
    num_rows: 323622
})


In [ ]:
ds_ko_qp = ds_ko.remove_columns([col for col in ds_ko.column_names if col not in ["anchor", "positive"]])
ds_en_qp = ds_en.remove_columns([col for col in ds_en.column_names if col not in ["anchor", "positive"]])

In [11]:
# label의 0번째 원소가 음수인 것만 따로 모음
ds_ko_neg_label0 = ds_ko.filter(lambda x: x["label"][0] < 0, num_proc=16)
print(f"ds_ko 전체: {len(ds_ko)}, label[0] < 0 인 것: {len(ds_ko_neg_label0)}")
ds_ko_neg_label0

Filter (num_proc=16): 100%|██████████| 1539300/1539300 [10:54<00:00, 2353.34 examples/s]


ds_ko 전체: 1539300, label[0] < 0 인 것: 1074991


Dataset({
    features: ['anchor', 'positive', 'negative_1', 'negative_2', 'negative_3', 'negative_4', 'negative_5', 'negative_6', 'negative_7', 'negative_8', 'negative_9', 'negative_10', 'negative_11', 'negative_12', 'negative_13', 'negative_14', 'negative_15', 'negative_16', 'negative_17', 'negative_18', 'negative_19', 'negative_20', 'negative_21', 'negative_22', 'negative_23', 'negative_24', 'negative_25', 'negative_26', 'negative_27', 'negative_28', 'negative_29', 'negative_30', 'negative_31', 'negative_32', 'label'],
    num_rows: 1074991
})

In [12]:
ds_ko_neg_label0[0]

{'anchor': '2020년 6월 23일부터 2021년 6월 8일까지 어디의 에스컬레이터를 교체하는 공사를 시행하지',
 'positive': '『노후 승강설비(에스컬레이터) 교체공사에 따른』 유형자산(선로설비) 불용건의 I 관련근거 승강기관리단-487호(2020.01.15.)『1~4호선에스컬레이터 분해정비및노후교체계획변경보고』 계약번호 2020060F0AD-00호(2020.06.23.)『이대역 노후 에스컬레이터 3,4호기 교체공사』 이대역 노후 에스컬레이터 3,4호기 교체공사 준공계 및 준공검사원 II 공사개요 공사명 : 이대역 노후 에스컬레이터 3,4호기 교체공사 - 계약자 : 수림엘리베이터(주) 대표 *** - 계약금액 : 금1,020,500,000원 (금일십억이천오십만원) - 공사기간 : 2020.06.23. 〜2021.06.08. - 사업대상 : 이대역 3,4호기 (2대) - 준공일 : 2021.05.25.',
 'negative_1': '우리동네키움센터 6호점 비상계단 설치 계획 III 추진 계획 공사기간 : 2021. 4월 초 ~ 5월 중순(착수일로부터 45일) 공사위치 : 6호점 예정지(구 작은도서관) 앞 옹벽 공사내용 - 도로경계기준 내에서 옹벽 일부 철거, 기존 옹벽 기초에 철근 연결하여 콘크리트 계단 연결 시공 옹벽 일부 철거, 계단 형틀 및 철근배근 , 콘크리트 타설(1~2주) 형틀해체, 되메우기, 옹벽 면처리 등(1주) 난간공사, 도장공사, 주변 정리 등(1~2주)',
 'negative_2': '인천 미추홀구는 3일 중회의실에서 한국철도공사와 도화역 에스컬레이터 설치사업 협약을 했다고 밝혔다. 도화역 에스컬레이터 설치사업은 도화역 1, 3번 외부 출입구 계단을 철거하고 양방향 에스컬레이터 4대를 설치하는 것이다. 구는 이달 중 실시설계를 거쳐 6월에 착공하고 내년 6월까지 완공할 계획이다. 사업비는 16억원으로 구와 한국철도공사(코레일)가 50%씩 분담한다. 도화역은 도화지구 개발에 따라 행정타운이 들어서고 아파트 입주가 시작하면서

In [14]:
from datasets import concatenate_datasets
ds_qp = concatenate_datasets([ds_ko_qp, ds_en_qp])
ds_qp
ds_qp.save_to_disk(os.path.join("/data_x/yjoonjang/PAPERS/Anchor-Embedding/DATA/QP", "train_ko_en"))


Saving the dataset (6/6 shards): 100%|██████████| 1862922/1862922 [00:27<00:00, 67786.55 examples/s] 
